In [11]:
#from pyspark import SparkConf,SparkContext
from pyspark.mllib.recommendation import Rating,ALS
from pyspark.sql import SparkSession
import sys

In [12]:
spark = SparkSession.builder.appName("Recommend").master("local[2]").getOrCreate()
movierdd = spark.read.option("header","true").csv("/home/maniram/big_data/movie/movies.csv")
ratingsrdd = spark.sparkContext.textFile("/home/maniram/big_data/movie/u.data")

In [13]:
def movieIdDictionary():
    movieNames = {}
    movielist = movierdd.select("movieId","title").collect()
    for i in range(movierdd.count()):
        movieNames[int(movielist[i].movieId)] = movielist[i].title 
    return movieNames

In [25]:

ratingsRDD = ratingsrdd.map(lambda l: l.split(',')) \
            .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2]))) \
            .cache()
print("Total samples : ",ratingsRDD.count())
ratingsRDD.take(5)



Total samples :  100004


[Rating(user=1, product=31, rating=2.5),
 Rating(user=1, product=1029, rating=3.0),
 Rating(user=1, product=1061, rating=3.0),
 Rating(user=1, product=1129, rating=2.0),
 Rating(user=1, product=1172, rating=4.0)]

In [26]:
print("Traing of Data Starts using ALS")
rank = 10
numIterations = 6
model = ALS.train(ratingsRDD,rank,iterations=numIterations)
userId = 1
print("Rating given by User Id : ",userId)
filterUserRating = ratingsRDD.filter(lambda x :x[0]==userId)
movieNamesDict = movieIdDictionary()
for rating in filterUserRating.collect():
    print (movieNamesDict[int(rating[1])] + ": " + str(rating[2]))

print("Top 10 recommendation for the user : \n ")

recommend_movie_list =[]

recommendMovie = model.recommendProducts(userId,10)
for recommendations in recommendMovie:
    print("Movie ",movieNamesDict[int(recommendations[1])]," Scores ",str(recommendations[2]))
    recommend_movie_list.append(movieNamesDict[int(recommendations[1])])


Traing of Data Starts using ALS
Rating given by User Id :  1
Dangerous Minds (1995): 2.5
Dumbo (1941): 3.0
Sleepers (1996): 3.0
Escape from New York (1981): 2.0
Cinema Paradiso (Nuovo cinema Paradiso) (1989): 4.0
Deer Hunter, The (1978): 2.0
Ben-Hur (1959): 2.0
Gandhi (1982): 2.0
Dracula (Bram Stoker's Dracula) (1992): 3.5
Cape Fear (1991): 2.0
Star Trek: The Motion Picture (1979): 2.5
Beavis and Butt-Head Do America (1996): 1.0
French Connection, The (1971): 4.0
Tron (1982): 4.0
Gods Must Be Crazy, The (1980): 3.0
Willow (1988): 2.0
Antz (1998): 2.0
Fly, The (1986): 2.5
Time Bandits (1981): 1.0
Blazing Saddles (1974): 3.0
Top 10 recommendation for the user : 
 
Movie  Business of Strangers, The (2001)  Scores  8.638791586728951
Movie  Ravenous (1999)  Scores  8.457530930387266
Movie  Hear My Song (1991)  Scores  7.842387948442017
Movie  Fabulous Baker Boys, The (1989)  Scores  7.583680653370565
Movie  Nowhere in Africa (Nirgendwo in Afrika) (2001)  Scores  7.561571477709581
Movie  How

In [27]:
print("Following movie should be recommended to user %d based user activity in the past\n\n\n"%userId)
for movieName in recommend_movie_list:
    print(movieName)
print("\n"*3,"Operation Finished")



Following movie should be recommended to user 1 based user activity in the past



Business of Strangers, The (2001)
Ravenous (1999)
Hear My Song (1991)
Fabulous Baker Boys, The (1989)
Nowhere in Africa (Nirgendwo in Afrika) (2001)
Howards End (1992)
Horse Feathers (1932)
Marvin's Room (1996)
Garden of the Finzi-Continis, The (Giardino dei Finzi-Contini, Il) (1970)
Trees Lounge (1996)



 Operation Finished
